In [5]:
%config IPCompleter.use_jedi = False
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import cvxpy as cp
import pandas as pd
np.set_printoptions(precision=4)

from pathlib import Path
fig_path = str(Path().absolute())+'/figures/hw3/'
print(fig_path)
data_path = str(Path().absolute())+'/hw3/data/'
print(data_path)

/home/zpyang/grad_courses/2021_spring/ece595_ml/figures/hw3/
/home/zpyang/grad_courses/2021_spring/ece595_ml/hw3/data/


## Exercise 2

In [14]:
train_cat = pd.read_csv(data_path+"train_cat.txt", delimiter=',', header=None)
train_cat = np.matrix(train_cat)
train_grass = pd.read_csv(data_path+"train_grass.txt", delimiter=',', header=None)
train_grass = np.matrix(train_grass)

(64, 9556)